# 0. 구글 드라이브 연동 및 필수 라이브러리 설치 및 로드

### 구글 드라이브 연동

In [61]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 필수 라이브러리 설치

In [62]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 필수 라이브러리 로드

In [131]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import re

In [81]:
# nltk.download()

### 폴더 경로 설정

In [64]:
path = '/content/gdrive/MyDrive/FINAL_SUBMIT/'

# 1. AI HUB 데이터 로드

### 데이터 로드 

In [65]:
train0 = pd.read_excel(path + '감성대화말뭉치(최종데이터)_Training.xlsx', engine='openpyxl')
validation0 = pd.read_excel(path + '감성대화말뭉치(최종데이터)_Validation.xlsx', engine='openpyxl')

### 두 데이터 합치기

In [66]:
aihub0 = pd.concat([train0, validation0], ignore_index=True)

In [67]:
print('train0의 sample 수({}) + validation0의 sample 수({}) = aihub0의 sample의 수({})'.format(train0.shape[0], validation0.shape[0], aihub0.shape[0]))

train0의 sample 수(40879) + validation0의 sample 수(5130) = aihub0의 sample의 수(46009)


# 2. 데이터 정제

### aihub 데이터에서 사용할 열 선택

In [68]:
for i in range(1,5):
  print('사람문장{}의 결측값 비율: {:.2f}%'.format(i, (aihub0['사람문장{}'.format(i)].isnull().sum()/aihub0.shape[0])*100))    # 사람문장4의 결측값 비율이 너무 높으므로 사용하지 않는다.

사람문장1의 결측값 비율: 0.00%
사람문장2의 결측값 비율: 0.00%
사람문장3의 결측값 비율: 24.07%
사람문장4의 결측값 비율: 96.47%


In [69]:
aihub0[['감정_대분류','사람문장1']].head(2)

,감정_대분류,사람문장1
0,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.
1,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.


In [70]:
aihub0[['감정_대분류','사람문장2']].head(2)

,감정_대분류,사람문장2
0,기쁨,아 지금 정말 신이 나.
1,불안,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.


In [71]:
aihub0[['감정_대분류','사람문장3']].head(3)    # 사람문장3은 대화의 뒷 부분이다 보니 감정 대분류와 맞지 않는 경우가 많다. 그러므로 사용하지 않는다.

,감정_대분류,사람문장3
0,기쁨,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.
1,불안,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.
2,당황,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.


In [72]:
aihub0[['감정_대분류','사람문장4']].head(2)

,감정_대분류,사람문장4
0,기쁨,NaN
1,불안,NaN


### 레이블 설정

In [73]:
aihub0['감정_대분류'].value_counts()    # 기쁨을 긍정(1), 나머지를 부정(0)으로 레이블 할 것이다.

불안     8235
분노     7793
슬픔     7776
기쁨     7571
상처     7440
당황     7192
불안        1
기쁨        1
Name: 감정_대분류, dtype: int64

### aihub0 -> aihub1

In [74]:
aihub1 = aihub0.copy()
aihub1 = aihub1[['감정_대분류', '사람문장1', '사람문장2']]

aihub1['label'] = 0
aihub1.loc[aihub1['감정_대분류']=='기쁨', 'label'] = 1
aihub1.loc[aihub1['감정_대분류']=='기쁨 ', 'label'] = 1

### 중복된 값 제거

In [75]:
print('aihub1의 샘플 수: {}\naihub1[\'사람문장1\']의 종류: {}\naihub1[\'사람문장2\']의 종류: {}'.format(aihub1.shape[0], aihub1['사람문장1'].nunique(), aihub1['사람문장2'].nunique()))
# 중복된 값이 존재함을 알 수 있다.

aihub1의 샘플 수: 46009
aihub1['사람문장1']의 종류: 44172
aihub1['사람문장2']의 종류: 43517


In [76]:
aihub1.drop_duplicates(subset=['사람문장1'], inplace=True, ignore_index=True)
aihub1.drop_duplicates(subset=['사람문장2'], inplace=True, ignore_index=True)

In [77]:
print('aihub1의 샘플 수: {}\naihub1[\'사람문장1\']의 종류: {}\naihub1[\'사람문장2\']의 종류: {}'.format(aihub1.shape[0], aihub1['사람문장1'].nunique(), aihub1['사람문장2'].nunique()))
# 중복된 값이 사라진 것을 볼 수 있다.

aihub1의 샘플 수: 43314
aihub1['사람문장1']의 종류: 43314
aihub1['사람문장2']의 종류: 43314


### aihub1 -> aihub2_1, aihub2_2

In [78]:
aihub2 = aihub1.copy()

In [79]:
aihub2_1 = aihub2[['사람문장1', 'label']]
aihub2_2 = aihub2[['사람문장2', 'label']]

### 여러 문장으로 구성된 샘플 처리

In [82]:
aihub2_1['nsent'] = aihub2_1['사람문장1'].map(sent_tokenize).map(len)
aihub2_2['nsent'] = aihub2_2['사람문장2'].map(sent_tokenize).map(len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [93]:
print('aihub2_1의 nsent(문장의 수):\n{}'.format(aihub2_1['nsent'].value_counts()))
print('-'*25)
print('aihub2_2의 nsent(문장의 수):\n{}'.format(aihub2_2['nsent'].value_counts()))

# 샘플이 1~6개의 문장들로 이루어진 것을 알 수 있다. 

aihub2_1의 nsent(문장의 수):
1    31883
2    11129
3      280
4       22
Name: nsent, dtype: int64
-------------------------
aihub2_2의 nsent(문장의 수):
1    27151
2    15197
3      893
4       63
5        9
6        1
Name: nsent, dtype: int64


In [95]:
aihub2_1.loc[aihub2_1['nsent']==2, '사람문장1'].head()

# 문장이 2개인 경우, 보통 '감정 + 그 이유'의 형식을 갖고 있다.
# 즉, 문장이 2개인 경우는 분리해서 사용하면 되겠다.

3         재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
8                친구들이 나를 괴롭혀. 부모님과 선생님께 얘기했는데도 믿어주지 않아.
10    새로 간 치과에서 조금 썩기만 한 이를 임플란트를 하라고 했어. 과잉진료는 안 한다더니.
13                      아내의 건강이 갑자기 악화됐어. 너무 걱정스럽고 불안해.
16            이번에도 고시에 불합격했어. 부모님께 무슨 소리를 들을지 벌써부터 두려워.
Name: 사람문장1, dtype: object

In [96]:
aihub2_1.loc[aihub2_1['nsent']==3, '사람문장1'].head()

# 문장이 3개인 경우부터는 모든 문장들이 동일한 감정을 나타내지 않거나 애매한 경우가 존재한다.
# 그러므로 문장이 3개 이상인 것들은 아예 제외하는 것이 낫다고 판단했다.

90         정년퇴직을 하게 되었어. 노후 준비를 다 해놔서 그런지 두렵지 않아. 너무 기뻐.
95                요즘 내 진로에 대해서 고민이 많아. 명확한 꿈이 없어. 혼란스러워.
187         오빠는 일이 없나 봐. 항상 혼자 방에 틀어박혀 있어. 그런 오빠가 좀 걱정돼.
368    항상 피로감은 있었지만 간암일 줄이야. 병원에서 죽을 수도 있다는데 참혹하고 화가 ...
636    내 주위에 괴롭힘을 당하는 친구가 있어. 분명 나에게 도움을 청할 거야. 난 그렇게...
Name: 사람문장1, dtype: object

###  aihub2_1, aihub2_2 ->  aihub3

In [98]:
aihub2_1.columns = ['document', 'label', 'nsent']
aihub2_2.columns = ['document', 'label', 'nsent']

# dataframe의 열 이름을 통일한다.

In [100]:
aihub3 = pd.concat([aihub2_1, aihub2_2], ignore_index=True)

In [106]:
aihub3 = aihub3.loc[aihub3['nsent']<=2]

# 문장이 3개 이상인 샘플들은 지운다.

### aihub3 -> aihub4

In [139]:
aihub4 = aihub3.copy()

In [140]:
aihub4 = aihub4[['document', 'label']]

### 두개의 문장으로 구성된 샘플을 두개의 샘플로 나눈다.(aihub4 -> aihub5)

In [141]:
# 문장 2개를 각각 하나의 문장으로 분리해줄 함수 정의
# 주의: dataframe의 인덱스가 오름차순읋 정렬되어 있어야 한다.
def trans_df(df):
  temp = pd.DataFrame({'document':[], 'label':[]})
  num = 0
  for i in tqdm(range(df.shape[0])):
    for s in sent_tokenize(df['document'][i]):
      temp.loc[num] = [s, int(df['label'][i])]
      num = num+1
  return temp

In [142]:
aihub4.reset_index(drop=True, inplace=True)

# 함수의 인풋 조건에 맞도록 인덱스를 오름차순으로 정렬시킨다.

In [143]:
aihub5 = trans_df(aihub4)

# 함수를 이용하여 데이터 프레임의 각 샘플들이 하나의 문장만을 갖도록 만든다.

100%|██████████| 85360/85360 [14:09<00:00, 100.50it/s]


### aihub5 -> aihub6

In [144]:
aihub6 = aihub5.copy()

In [145]:
aihub6['document'] = aihub6['document'].map(lambda sent: re.sub('[^ 가-힣]', '', sent))

In [148]:
aihub6['label'] = aihub6['label'].astype('int64')

### aihub6 -> aihub7(중복된 값 제거)

In [153]:
aihub7 = aihub6.copy()

In [155]:
print('aihub7의 샘플 수: {}\naihub7[\'document\']의 종류: {}'.format(aihub7.shape[0], aihub7['document'].nunique()))
# 중복된 값이 존재함을 알 수 있다.

aihub7의 샘플 수: 111686
aihub7['document']의 종류: 103945


In [156]:
aihub7 = aihub7.drop_duplicates(subset=['document'], ignore_index=True)

In [158]:
print('aihub7의 샘플 수: {}\naihub7[\'document\']의 종류: {}'.format(aihub7.shape[0], aihub7['document'].nunique()))
# 중복된 값이 지워진 것을 볼 수 있다.

aihub7의 샘플 수: 103945
aihub7['document']의 종류: 103945


# 3. 데이터 내보내기

In [159]:
aihub7.to_excel(path + 'aihub_data_preprocessed.xlsx', index=False)